The US Department of Health and Human Services provides federal-level collection and publishing of COVID-19 testing and patient outcome data. To better understand the current state of COVID-19 testing in the US, we’d like you to create a Python project and documentation for the following metrics:
•	The total number of PCR tests performed as of yesterday in the United States.
•	The 7-day rolling average number of new cases per day for the last 30 days.
•	The 10 states with the highest test positivity rate (positive tests / tests performed) for tests performed in the last 30 days.



Data ingestion
•	It is assumed data is in a json file with the following structure:        [{"state":"xx","state_name":"xxx","state_fips":"xx","fema_region":"xxx…","overall_outcome":"xxx…","date":"xxxx…","new_results_reported":"xxx…","total_results_reported":"xxx…"},{….}]
•	Data could be read from a given location


In [234]:
# load pandas and json modules                                                                                               
import pandas as pd
import json

Source location and file name definition

In [235]:
#source_location = 'https://healthdata.gov/resource/'
source_location = 'C:/Users/arodriguezbe/OneDrive - Suncor Energy Inc/'
#source_file = 'j8mb-icvb.json'
source_file = 'COVID-19_Diagnostic_Laboratory_Testing__PCR_Testing__Time_Series.csv'

In [5]:
df_covid = pd.read_json(source_location + source_file)

In [236]:
df_covid = pd.read_csv(source_location + source_file, index_col=None)

To check data formats

In [237]:
df_covid['date']= df_covid['date'].astype(str)
df_covid['date']= df_covid['date'].astype('datetime64[ns]')

To order data set by "date"

In [238]:
df_covid.sort_values(by=['date'], inplace = True)
df_covid = df_covid.reset_index(drop=True)  

•	The total number of PCR tests performed as of yesterday in the United States.

In [239]:
df_pcr_totals = df_covid.groupby('date').new_results_reported.agg(df_pcr_totals = ('sum'))
df_pcr_totals = df_pcr_totals.reset_index()  

In [240]:
print('The total number of PCR tests performed as of yesterday in the United States :',df_pcr_totals['df_pcr_totals'].iloc[len(df_pcr_totals)-2], 'cases')

The total number of PCR tests performed as of yesterday in the United States : 143630 cases


•	The 7-day rolling average number of new cases per day for the last 30 days.

In [229]:
date_list = df_covid.date.unique()
date_list.sort()
days_list = date_list[-30:]
last_30days = pd.DataFrame({'date':days_list})

In [230]:
df_rolling_last30days = pd.merge(last_30days, df_pcr_totals, on=['date'], how = "left")

In [231]:
df_pcr_rolling_avg = df_rolling_last30days['df_pcr_totals'].rolling(7).mean()
df_pcr_rolling_avg = df_pcr_rolling_avg.reset_index() 
df_pcr_rolling_avg = df_pcr_rolling_avg.rename(columns={'df_pcr_totals':'7day_rolling_avg'})
df_pcr_rolling_avg['day'] = df_pcr_rolling_avg['index']+1
df_pcr_rolling_avg =df_pcr_rolling_avg[['day','7day_rolling_avg']]


In [233]:
print('The 7-day rolling average number of new cases per day for the last 30 days.\n',df_pcr_rolling_avg)

The 7-day rolling average number of new cases per day for the last 30 days.
     day  7day_rolling_avg
0     1               NaN
1     2               NaN
2     3               NaN
3     4               NaN
4     5               NaN
5     6               NaN
6     7     869699.285714
7     8     866502.142857
8     9     848681.285714
9    10     828982.428571
10   11     823619.428571
11   12     797666.571429
12   13     750483.714286
13   14     699912.142857
14   15     651569.714286
15   16     610897.714286
16   17     584229.142857
17   18     574411.285714
18   19     522510.857143
19   20     482608.857143
20   21     468090.142857
21   22     479252.571429
22   23     489042.571429
23   24     492203.714286
24   25     494018.285714
25   26     528494.000000
26   27     539187.285714
27   28     504365.857143
28   29     419343.857143
29   30     341379.857143


In [221]:
print('The 7-day rolling average number of new cases per day for the last 30 days:\n')
print(df_pcr_rolling_avg)


The 7-day rolling average number of new cases per day for the last 30 days:

    index  df_pcr_totals  day
0       0            NaN    1
1       1            NaN    2
2       2            NaN    3
3       3            NaN    4
4       4            NaN    5
5       5            NaN    6
6       6  869699.285714    7
7       7  866502.142857    8
8       8  848681.285714    9
9       9  828982.428571   10
10     10  823619.428571   11
11     11  797666.571429   12
12     12  750483.714286   13
13     13  699912.142857   14
14     14  651569.714286   15
15     15  610897.714286   16
16     16  584229.142857   17
17     17  574411.285714   18
18     18  522510.857143   19
19     19  482608.857143   20
20     20  468090.142857   21
21     21  479252.571429   22
22     22  489042.571429   23
23     23  492203.714286   24
24     24  494018.285714   25
25     25  528494.000000   26
26     26  539187.285714   27
27     27  504365.857143   28
28     28  419343.857143   29
29     29  341379.85714

•	The 10 states with the highest test positivity rate (positive tests / tests performed) for tests performed in the last 30 days.

In [202]:
df_covid_last_30days = pd.merge(last_30days, df_covid, on=['date'], how = "left")
df_pcr_totals = df_covid_last_30days.groupby(['state_name']).new_results_reported.agg(tests_performed = ('sum'))
df_pcr_totals = df_pcr_totals.reset_index()

In [203]:
df_pcr_positive1 = df_covid_last_30days[(df_covid_last_30days.overall_outcome == "Positive")].copy() 
df_pcr_positive = df_pcr_positive1.groupby(['state_name']).new_results_reported.agg(positive_tests_performed = ('sum'))


In [204]:
df_positivity_rate = pd.merge(df_pcr_totals, df_pcr_positive, on=['state_name'], how = "left")
df_positivity_rate.sort_values(by=['state_name'], inplace = True)


In [206]:
df_positivity_rate['positivity_rate'] = df_positivity_rate['positive_tests_performed']/df_positivity_rate['tests_performed']
df_positivity_rate = df_positivity_rate[['state_name','positivity_rate']]
df_positivity_rate['positivity_rate'] = df_positivity_rate['positivity_rate'].fillna(0)

In [207]:
df_positivity_rate.sort_values(by=['positivity_rate'], inplace = True)
df_positivity_rate = df_positivity_rate.reset_index(drop=True)  
df_states = df_positivity_rate.tail(10)
df_states = df_states.reset_index()
print(df_states)

   index           state_name  positivity_rate
0     44             Virginia         0.180712
1     45             Delaware         0.185343
2     46              Arizona         0.191249
3     47              Florida         0.191401
4     48               Hawaii         0.205829
5     49                 Utah         0.222829
6     50               Nevada         0.254257
7     51  U.S. Virgin Islands         0.285941
8     52          Puerto Rico         0.290394
9     53                 Iowa         1.000000
